In [44]:
import numpy as np
import math

def my_split(s):
    block_start = False
    start_index = 0
    ret_list=[]
    for i, c in enumerate(s):
        if block_start==False:
            if c==',':
                ret_list.append(s[start_index:i])
                start_index=i+1
            elif c=='"':
                block_start=True
                start_index = i
        else:
            if c=='"':
                block_start=False
    if s[-1]!=',':
        ret_list.append(s[start_index:])
    return ret_list

def split_len(data_list):
    len_list=[]
    for e in data_list:
        len_list.append(len(e))
    print(set(len_list))
    if len(set(len_list))>1:
        for i in set(len_list):
            print(i, len_list.count(i))

In [45]:
csv_data = []
with open('전국로컬푸드인증정보표준데이터.csv',encoding='EUC-KR') as f:
    for line in f:
         #print(line)
        csv_data.append(my_split(line[:-1]))  # /n 없앰
        
print(csv_data[:3])
split_len(csv_data)


[['로컬푸드인증번호', '인증유효시작일자', '인증유효종료일자', '인증취소일자', '사업장도로명주소', '사업장지번주소', '생산지도로명주소', '생산지지번주소', '품목명', '재배면적', '재배규모', '인증부여일자', '사업자명', '사업자등록번호', '전화번호', '관리기관명', '데이터기준일자', '제공기관코드', '제공기관명'], ['HYG-162', '2021-09-01', '2031-09-01', '', '경상남도 함양군 수동면 산업단지길 209+경상남도 함양군 수동면 우명하교로 174', '', '경상남도 함양군 병곡면 다볕길 94', '', '칡청+달맞이꽃씨유', '', '122', '2021-09-01', '청운당농산', '', '070-7779-2122', '경상남도 함양군청', '2022-11-23', '5460000', '경상남도 함양군'], ['HYG-163', '2021-09-01', '2031-09-01', '', '경상남도 함양군 수동면 산업단지길 209+경상남도 함양군 수동면 우명하교로 174', '', '경상남도 함양군 수동면 사근산성길 35-4', '', '굼벵이환', '', '123', '2021-09-01', '굼찬농장', '', '070-7779-2122', '경상남도 함양군청', '2022-11-23', '5460000', '경상남도 함양군']]
{19}


In [46]:
for e in enumerate (csv_data[0]): #csv_data[0] , header의 row 확인
    print(e)

(0, '로컬푸드인증번호')
(1, '인증유효시작일자')
(2, '인증유효종료일자')
(3, '인증취소일자')
(4, '사업장도로명주소')
(5, '사업장지번주소')
(6, '생산지도로명주소')
(7, '생산지지번주소')
(8, '품목명')
(9, '재배면적')
(10, '재배규모')
(11, '인증부여일자')
(12, '사업자명')
(13, '사업자등록번호')
(14, '전화번호')
(15, '관리기관명')
(16, '데이터기준일자')
(17, '제공기관코드')
(18, '제공기관명')


In [47]:
np_data = np.array(csv_data)
#print(np_data.shape) # data 갯수
np_data = np_data[:,4:9] # 사업장도로명주소 ~ 품목명까지
print(np_data[0])

['사업장도로명주소' '사업장지번주소' '생산지도로명주소' '생산지지번주소' '품목명']


In [49]:
def process_product_names(s):
    r = [ ]
    remove_after = ['(','외']
    split_at = ['+',',']
    for c in remove_after:
        if c in s:
            s = s[:s.index(c)]
            
    for c in split_at:
        if c in s:
            r = s.split(c)
            
    if len(r) == 0:
        r.append(s)
    

    return r

sample1 = 'productA+productB'
sample2 = 'productA,productB'
sample3 = 'product(A+B)'
sample4 = 'product외 abc'
sample5 = 'product'
assert process_product_names(sample1) == ['productA','productB']
assert process_product_names(sample2) == ['productA','productB']
assert process_product_names(sample3) == ['product']
assert process_product_names(sample4) == ['product']
assert process_product_names(sample5) == ['product']


In [54]:
area_name_list = []
product_list = []
all_data = []
for sample in np_data[1:]:
    area_name = '' #비어있는 문자열
    for i in sample[:-1]:
        i = i.strip(' "')
        if len(i) > 0 : # 빈칸이 아니면,
            area_name = ' '.join(i.split()[:2])
            #area_name_list.append(area_name)
            break
    #product_list.append(process_product_names(sample[-1]))
    for p in process_product_names(sample[-1]):
        all_data.append([area_name,p])
        
        
#print(area_name_list[:3],len(area_name_list))
#print(product_list[:3],len(product_list))

In [60]:
#print(all_data[:10])
fw_area_name = []
fw_products = []
for v in all_data:
    if v[0] in fw_area_name:
        i = fw_area_name.index(v[0])
        fw_products[(i)].append(v[1])
    else:
        fw_area_name.append(v[0])
        fw_products.append([v[1]])
#print(fw_area_name[:3],len(fw_area_name))
#print(fw_products[:3],len(fw_products))
fw_dict = dict(zip(fw_area_name,fw_products))
print(fw_dict)
print(fw_dict['경상남도 함양군'])
print(len(fw_dict['경상남도 함양군']))

{'경상남도 함양군': ['칡청', '달맞이꽃씨유', '굼벵이환', '메론', '차요태', '열매마', '포도', '꿀', '포도', '사과', '샤인머스켓', '미숫가루', '밀가루', '쌀', '사과', '알밤', '한라봉', '레드향', '천혜향', '사과', '서복태', '함양애주', '지리산애주', '키고리', '스카프', '가방', '두건', '로메인 상추', '더덕주', '블루베리', '호두', '새싹인삼', '유정란', '한과', '도라지', '곶감', '감말랭이', '딸기', '고로쇠 수액', '지리산도라지와돌배', '곶감', '말랭이', '사과', '딸기', '복수박', '두릅', '고사리', '취나물', '누른호박', '배', '즙류', 'ABC주스', '우슬진액', '꿀류', '차류', '미나리', '두릅', '취나물', '머위즙', '삼백초즙', '블루베리', '신령호두', '부각류', '블루베리', '가지', '애호박', '사과', '차류', '아스파라거스', '담솔', '복분자', '솔송주', '고구마', '사과', '도마', '주걱', '약초류', '유기농계란', '엉겅퀴', '오가피', '죽염류', '된장', '치약', '국화분재', '표고버섯', '도라지', '부각류', '그린하미과', '컬리플라워', '생강청', '산골편강', '도라비배청', '보리새싹', '레몬머틀', '배', '고구마', '오란다', '조청산자', '김부각', '강정류', '꿀류', '곶감', '대파', '양상추', '곶감', '사과', '사과즙', '오미자청', '도라지배즙', '고구마', '무말랭이', '시래기', '사과', '사과즙', '반쯤깐밤', '산삼캔디', '지리산명월초 티백차', '표고버섯', '우산고로쇠 수액', '본래족발스페셜세트', '이불', '베개', '목화솜', '꾸찌봉차', '꾸지뽕진액', '오미자즙', '양파즙', '매실원액', '개복숭아원액', '솔순원액', '고로쇠 수액', '달걀', '꿀', '화분', '아스파라거스', '꿀

In [62]:
rev_dict = dict()
for v in all_data:
    if v[1] not in rev_dict.keys():
        rev_dict[v[1]] = {v[0]}
    else:
        rev_dict[v[1]].add(v[0])
print(len(rev_dict))
print(rev_dict['사과'])

1424
{'충청남도 당진시', '전라북도 완주군', '영월군 영월읍', '영월군 중동면', '충청북도 옥천군', '영월군 주천면', '강원도 원주시', '경기도 수원시', '강원도 영월군', '충청남도 아산시', '경상남도 함양군', '영월군 남면', '영월군 북면', '강원도 춘천시', '경기도 화성시', '충청북도 괴산군', '전라남도 순천시', '충청남도 서산시'}


In [64]:
p_list = []
for v in all_data:
    p_list.append(v[1])
p_set = set(p_list)
count_list = []
for v in p_set:
    count_list.append((v,p_list.count(v)))
x = sorted(count_list,key=lambda x:x[1],reverse = True)
print(x)

[('고추', 541), ('대파', 348), ('상추', 289), ('배추', 249), ('고구마', 234), ('딸기', 225), ('감자', 219), ('포도', 217), ('복숭아', 207), ('감말랭이', 187), ('블루베리', 184), ('무', 179), ('쪽파', 150), ('부추', 150), ('양파', 148), ('오이', 136), ('토마토', 133), ('감', 131), ('시금치', 129), ('마늘', 119), ('가지', 119), ('사과', 99), ('열무', 96), ('대추', 89), ('표고버섯', 83), ('당근', 81), ('도라지', 66), ('생강', 65), ('호박', 65), ('양배추', 61), ('깻잎', 59), ('고사리', 59), ('아로니아', 56), ('콩', 56), ('땅콩', 55), ('배', 54), ('취나물', 52), ('고구마순', 50), ('쌀', 48), ('자두', 45), ('두릅', 45), ('갓', 44), ('옥수수', 43), ('방울토마토', 43), ('아욱', 43), ('단호박', 42), ('파', 41), ('머위', 40), ('애호박', 38), ('서리태', 38), ('아스파라거스', 37), ('호박잎', 35), ('물김', 34), ('무청시래기', 33), ('얼갈이배추', 32), ('고구마줄기', 32), ('수박', 31), ('곶감', 30), ('냉이', 30), ('알타리무', 30), ('들깨', 30), ('돼지감자', 30), ('매실', 28), ('쑥갓', 27), ('밤', 27), ('복숭아 ', 27), ('풋마늘', 23), ('노각', 22), ('참깨', 22), ('달래', 22), ('단감', 20), ('여주', 20), ('고들빼기', 18), ('꽈리고추', 18), ('참나물', 18), ('메론', 18), ('다래', 17), ('더덕', 17),